## Introduction


This notebook contains workflows to parameterize a system (methanol) for molecular dynamics simulations with OpenMM and [MPID plugin](https://github.com/andysim/MPIDOpenMMPlugin).

### Dependencies
- OpenMM > 7
- MPID Plugin [installation](https://andysim.github.io/MPIDOpenMMPlugin/building)
- openmmforcefields
- parmed

In [1]:
import os
import numpy as np
from openff.toolkit import ForceField
from pkg_resources import resource_filename

from factorpol.parameterization import parameterize_molecule
from factorpol.utilities import BondChargeCorrections, Polarizability

cwd = os.getcwd()

off_forcefield = ForceField(resource_filename(
    "factorpol", os.path.join("data", "off_examples.offxml")
))

/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


### Prepare force field
Load previously derived polarizabilities and BCC-dPol parameters. For demonstration reason, they are not well defined and should only be used as examples.

In [2]:
polarizability = Polarizability(data_source="ret_alphas.csv")

In [3]:
bcc_dpol_library = BondChargeCorrections(data_source="ret_bccs.csv")

Use OpenFF Sage force field for valence and van der Waals parameters. GAFF is also available by specifying `ff_name="gaff"`

In [4]:
parm = parameterize_molecule(
    smile="CO",
    ff_name="sage",
    polarizability=polarizability,
    BCCLibrary=bcc_dpol_library,
    output_path=os.path.join(cwd, "data_mpid"),
    off_forcefield=off_forcefield,
)

### Create an OpenMM system
```
forcefield = ForceField("data_mpid/forcefield.xml")
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    polarization="direct",
    nonbondedCutoff=8 * angstrom,
    constraints=HBonds,
    defaultTholeWidth=8,
)
```

Print out the generated force field file:

In [5]:
print(parm["forcefield.xml"])

<ForceField>
  <Info>
    <DateGenerated>2023-03-30</DateGenerated>
  </Info>
  <AtomTypes>
    <Type element="C" name="C1" class="C1" mass="12.01078"/>
    <Type element="O" name="O1" class="O1" mass="15.99943"/>
    <Type element="H" name="H1" class="H1" mass="1.007947"/>
    <Type element="H" name="H2" class="H2" mass="1.007947"/>
  </AtomTypes>
  <Residues>
    <Residue name="UNK">
      <Atom name="C1x" type="C1" charge="0.0"/>
      <Atom name="O1x" type="O1" charge="0.0"/>
      <Atom name="H1x" type="H1" charge="0.0"/>
      <Atom name="H2x" type="H1" charge="0.0"/>
      <Atom name="H3x" type="H1" charge="0.0"/>
      <Atom name="H4x" type="H2" charge="0.0"/>
      <Bond atomName1="C1x" atomName2="O1x"/>
      <Bond atomName1="C1x" atomName2="H1x"/>
      <Bond atomName1="C1x" atomName2="H2x"/>
      <Bond atomName1="C1x" atomName2="H3x"/>
      <Bond atomName1="O1x" atomName2="H4x"/>
    </Residue>
  </Residues>
  <HarmonicBondForce>
    <Bond class1="C1" class2="O1" length="